In [3]:
import numpy as np

# Overview of functions

**Class: Scoreboard** <br>
Keeps track of the scores for all players. Basically the paper you write results on. Can also count the scores. <br>

Functions: 
- init : initialises the game with zero points for all players
- count_scores: can at all times return the current scores for all players
- update: updates the scoreboard by taking which player has chosen which category and how many points they scored are added to the scorebboard (equivalent to writing down points after each turn)
- display: print the scoreboard so a human player can visualise the state of the game

**Class: Player** <br>
There will be three types of players, i.e. three classes:
- Human player: allows for a human to play bby inputting choices through the keyboard
- Gollum: a greedy player
- YAS_Bot: Yatzy automatic system, a strategic player <br>

Functions: <br>
- choose_kept_dice: takes the scoreboard (the object), player number(the column index of the player in scoreboard), n_rolls (how many times the dice has been rolled this round: 1 or 2. Not 3 since we will use the other method in that case) and dice_state (array with the pips for each dice rolled or kept), and returns a keep mask with which dice to keep and not reroll, i.e. if it returns [1,0,0,0,0] the first dice is not rerolled and the rest are rerolled: dice_state[~keep_mask] = np.random.choice()
- choose_category: usually this will only be used after the last roll of dice, but if the game mode is set to log it can be used after every roll. This class method takes in the same inputs as the last one (maybe except for roll no.) and returns a choice of which dice to use (a use_mask) and which category to make them count in (a string)

**Class: Dice** <br>
<br>
Functions: 
- roll: 
- count_points: count the number of points for one category given an array of dice

**Class: Game** <br>
<br>
Functions: 


In [129]:
class Scoreboard:
    """
    Class keeping the scores of all players. Divided into lower and upper score and a total score vector. 
    The upper score are ones, twos, etc. The lower score are pairs, three of a kind, full house
    
    Say you roll 2 ones and want to use it as ones. At score, n_players there will be a 2.
    """
    
    # We initialise the score values with zeros until scores are filled in. 
    # If a turn is given up we fill with Nan
    def __init__(self, n_players):
        
        # Define number of players
        self.n_players = n_players
        
        # Start scores at zeros
        self.scores = np.zeros(n_players, dtype=int)
        
        # Rows: ones, twos, threes, fours, fives, sixes
        self.upper_scores = np.zeros(shape=(6, n_players), dtype=int)
        self.upper_names = np.array(['1s',   '2s',     '3s',    '4s',   '5s',   '6s'])
        
        # Rows: one_pair, two_pairs, three_of_a_kind, four_of_a_kind, small_straight, large_straight
        #.      full_house, chance, yatzy
        self.lower_scores = np.zeros(shape=(9, n_players), dtype=int)
        self.lower_names = np.array(['1 pair', '2 pairs',   '3 of a kind',  '4 of a kind', 'Small Straight', \
                            'Large Straight', 'Full House',  'Chance', 'Yatzy'])
        
        # Define bonus from upper scores
        self.bonus_cutoff = 63
        self.bonus = 50
    
    def count_scores(self):
        """
        Method to count scores
        """
 
        # Count upper scores (n_players, total scores for upper)
        upper_sums = np.sum(self.upper_scores, axis=0)
        
        # Assign bonus accordingly (n_players, bonuses=0 or 50)
        bonuses = ( upper_sums >= self.bonus_cutoff ) * self.bonus
        
        # Count lower scores (n_players, total scores for lower)
        lower_sums = np.sum(self.lower_scores, axis=0)
        
        # Define final scores
        self.scores = upper_sums + bonuses + lower_sums
        
    def update(self, player_turn, scoreboard_category, score_amount):
        """
        Updates the players scoreboard given a choice on the scoreboard and a score
        """
        # Update if choice is in upper
        if scoreboard_category in self.upper_names:
            
            # Index to update
            idx = self.upper_names.index(scoreboard_category)

            # Update the score of that category
            self.upper_scores[idx, player_turn] = score_amount
        
        # Update if choice is in lower
        elif scoreboard_category in self.lower_names:

            # Index to update
            idx = self.lower_names.index(scoreboard_category)

            # Update the score of that category
            self.lower_scores[idx, player_turn] = score_amount
            
        else:
            raise ValueError("Not a valid scoreboard category.")

        # Count score in new scoreboard
        self.count_scores()
        
    def display(self, player_names=None):
        """
        Prints the scoreboard so a Human player can see it.
        """
        
        # Print a line
        n_lines = (self.n_players + 1)*15
        print(n_lines*'-')
        
        # Make player names in case not given
        if not player_names:
            player_names = [f'Player {i+1}' for i in range(self.n_players)]
        
        # Print player names
        print('\t \t'+'\t'.join(player_names) )
        
        # Line
        print(n_lines*'-')
        
        # Upper box
        for i in range(len(self.upper_names)):
            print(f'{self.upper_names[i]} \t \t' + '\t \t'.join(self.upper_scores[i,:].astype(str)) )
            
        # Lower box
        for i in range(len(self.lower_names)):
    
            # Check how many tabs to put after name of category, depends on len of name
            if len(self.lower_names[i]) // 7 == 0:
                print(f'{self.lower_names[i]} \t \t' + '\t \t'.join(self.lower_scores[i,:].astype(str)) )
        
            else:
                print(f'{self.lower_names[i]} \t' + '\t \t'.join(self.lower_scores[i,:].astype(str)) )
            
        # Total Score
        print(n_lines*'-')
        print('Total Score \t'+'\t \t'.join(self.scores.astype(str)) )
        print(n_lines*'-')

In [ ]:
######## DEFINITIONS OF CHECKS ###########
# Generate for the upper classes. Check if all chosen dice are of the chosen category.
def uppers_gen(i):
    # Closure function to make functions that match numbers
    def upper(dice_states, choice):
        # Check that all the dice in dice_states have the value i
        return (np.asarray(dice_states[choice]) == i).all()
    return upper

# Small and large straight checks if there is 5 unique elements in dice_states and either 6 (for small) or 1 (for large) 
# is NOT in dice states
def small_straight(dice_states, choice):
    if np.sum(choice) != 5:
        return False
    return len(np.unique(dice_states)) == 5 and 6 not in dice_states

def large_straight(dice_states, choice):
    if np.sum(choice) != 5:
        return False
    else:
        return len(np.unique(dice_states)) == 5 and 6 not in dice_states

# Check if the dice are of some structure. For example 3 of a kind, 2 pairs og house 
def of_a_kind(amounts):
    """
    Check if dice_states fulfill some amounts.
    Amounts can be a number or a list of numbers 
    """
    def check(dice_states, choice):
        # Check that the sum of chosen dice are equal to the number required
        if np.sum(choice) != np.sum(amounts):
            return False
        else:
            # Make a list of the counts of uniques
            uni, uni_counts = np.unique(dice_states[choice], return_counts = True)

            # If the sorted list of unique counts match the desired amounts, return True.
            if (np.sort(uni_counts) == np.asarray(amounts)).all():
                return True
            else:
                return False
    return check

# Chance will always return True, unless no dice are selected.
def chance(dice_states, choice):
    return np.sum(choice) > 0

# Collect all the functions in a dictionairy. 
validity_dict = {
    "1s":               uppers_gen(1), 
    "2s":               uppers_gen(2),
    "3s":               uppers_gen(3),
    "4s":               uppers_gen(4),
    "5s":               uppers_gen(5),
    "6s":               uppers_gen(6),
    '1 pair':           of_a_kind(2), 
    '2 pairs':          of_a_kind([2, 2]),   
    '3 of a kind':      of_a_kind(3),  
    '4 of a kind':      of_a_kind(4), 
    'Small Straight':   small_straight, 
    'Large Straight':   large_straight,
    'Full House':       of_a_kind([2, 3]), 
    'Chance':           chancen,
    'Yatzy':            of_a_kind(5)}

class Dice:
    """
    Dice class. Made to keep track of a set of five dice. 

    Main methods are roll, which takes an iterable of five booleans and updates the dice states.

    Further it is used to calculate the score from a given choice of category as well as the dice used to form it. 
    """

    def __init__(self):
        """
        Intialize dice 
        """

        # Start an empty array
        self.dice_rolls = np.zeros(5, dtype = int)

        # Roll without keeping any dice
        self.roll(np.zeros(5, dtype = bool))

    def roll(self, keep_mask):
        """
        Update the dice_rolls of the class given a mask to keep
        """
        # Replace the dice, that we decided not to keep with new rolls
        self.dice_rolls[~keep_mask] = np.random.randint(1, 7, size = np.sum(~keep_mask))

    def count_points(self, choice_of_category = None, choice_of_dice = np.array([False] * 5)):
        """
        Check if a choice of category is valid and return the score. 
        Return np.nan if the choice is not valid.
        """
        # Convert choice_of_dice to array if it is not
        choice_of_dice = np.asarray(choice_of_dice)

        # Check if the move is valid using the dictionairy 
        valid_move = validity_dict[choice_of_category](self.dice_rolls, choice_of_dice)

        # If yatzy is called and it is a valid move, return 50
        if valid_move and choice_of_category == "Yatzy":
            return 50 # ADD HERE IF WE COUNT THE EYES 
        # If we have a valid choice. return the sum chosen dice
        elif valid_move:
            return np.sum(self.dice_rolls[choice_of_dice])
        # If move was not valid, return np.nan
        else:
            return np.nan


In [214]:
# Function that know which categories are still available
def legal_categories(current_scoreboard, player_number):
    
    # Extract indices of which categories have been filled
    legal_upper_mask = ( current_scoreboard.upper_scores[:,player_number-1] == 0 )
    legal_lower_mask = ( current_scoreboard.lower_scores[:,player_number-1] == 0 )

    # Create mask of available categories 
    legal_categories = np.concatenate(( current_scoreboard.upper_names[legal_upper_mask], \
                                         current_scoreboard.lower_names[legal_lower_mask] ))
    
    return legal_categories

def is_mask_legal(mask):
    if len(mask) == 5:
        return ( np.unique(mask) == np.array([0, 1]) ).all()

In [ ]:
class Human_player:
    """
    Class that allows a Human player to play by using the keyboard.
    """
    
    # We initialise the player by letting it know our name
    def __init__(self, name='Human Player'):
        
        self.name = name
        print(f'Welcome to Yatzy, {name}!')
    
    # We let the player make a choice with the rolled dice
    def keep_dice(self, current_scoreboard, player_number, n_rolls, dice_state):
        """
        For now doesnt actually use current scoreboard and n_rolls
        """
        
        # Show the player the dice
        print(f'You have rolled: {dice_state}')
        
        # Let player type a mask
        mask_str = input('Enter which dice you want to keep on the form \'1,0,0,0,0\' ' + '\n' +
                         'where 1 signals a kept die and 0 a die to be rerolled [q to quit]: \n')
        
        # Check if player has quit
        if mask_str == 'q':
            raise KeyboardInterrupt('Player has forfeitet the game.')
        
        # Convert mask to list with integers
        keep_mask = [ int(mask_str.split(',')[i]) for i in range(5) ]
        
        # Keep on checking that the mask is valid
        while not is_mask_legal(keep_mask):
            
            # Let player reenter which dice to keep
            print('The input is not a valid mask. Try again.')
            mask_str = input('Enter which dice you want to keep on the form \'1,0,0,0,0\' ' + '\n' +
                             'where 1 signals a kept die and 0 a die to be rerolled [q to quit]: \n')
            
            # Check if player has quit
            if mask_str == 'q':
                raise KeyboardInterrupt('Player has forfeitet the game.')
                
            # Convert mask to list with integers
            keep_mask = [ int(mask_str.split(',')[i]) for i in range(5) ]
        
        keep_mask = np.array(keep_mask, dtype=bool)
        
        return keep_mask
        
        
    # We let the player end a turn
    def choose_category(self, current_scoreboard, player_number, n_rolls, dice_state):
        
        """
        PLAN:
        
        1) First choose category? (allow for quiting?)
        2) check if valid if not show legal categories
        3) which dice to use
        4) check if valid mask if not ask again
        5) confirm choice if it will be a streg
        
        """
        
        
        # Show the player the dice
        print(f'You have rolled: {dice_state}')
        
        # Let player type a mask
        mask_str = input('Enter which dice you want to use on the form \'1,0,0,0,0\' ' + '\n' +
                         'where 1 signals a kept die and 0 a die to be rerolled [q to quit]: \n')
        
        
        #return category, use_mask
        

In [ ]:
# Test it
Kimi = Human_player('Johann')
board = Scoreboard(1)
dice_set = np.random.randint(1, 7, size = 5)

for i in range(2):
    keep_mask = Kimi.keep_dice(current_scoreboard = board, player_number=1, n_rolls=0, dice_state=dice_set)
    dice_set[~keep_mask] = np.random.randint(1, 7, size = np.sum(~keep_mask))
    
dice_set

Welcome to Yatzy, Johann!
You have rolled: [3 1 1 2 1]
